In [285]:
import gspread
import torch
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

In [286]:
credentials_path = '../riasec-model-1-c1119188e695.json' 

spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1x4e-yyXWHMZW9ZXiOb_oUnZLHukOWg1cc0Qf5jL2aLM/edit?usp=sharing'

scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
gc = gspread.authorize(credentials)

spreadsheet = gc.open_by_url(spreadsheet_url)

worksheet = spreadsheet.get_worksheet(0)

data = worksheet.get_all_values()

df = pd.DataFrame(data[1:], columns=data[0])

In [287]:
df

,Позначка часу,"Коли я отримую подарунки, я більше ціную практичні та корисні речі, ніж щось декоративне",Моя енергія ніколи не вичерпується,"Теоретичні ідеї та дилеми – це не те, на що варто витрачати багато часу\n(на скільки Ви погоджуєтесь з твердженням)",Чи вам подобається займатися фізичною активністю та спортом?,Чи цікавить вас робота з різноманітними інструментами?,Чи вам подобається працювати на відкритому повітрі?,Чи вам цікаво розв'язувати складні математичні задачі?,Чи любите ви читати наукові книги або журнали?,Мені завжди цікаво почути унікальні точки зору на речі,...,Я чудово вмію змусити інших підтримати мої ідеї\n\n,Я легко можу змінити думки оточуючих щодо якоїсь теми,Чи вам подобається приймати важливі рішення та вирішувати проблеми?,Чи берете ви участь у лідерських або підприємницьких ініціативах?,Чи вам подобається порядок і систематичність в роботі?,"Я завжди докладаю максимальних зусиль, щоб уникнути помилок\n\n",Чи легше вам працювати за чіткими правилами і встановленими стандартами?,Чи цікавитеся ви організаційними аспектами роботи?,"Я віддаю перевагу дотриманню правил, ніж імпровізації",Я легко концентруюсь на задачах які виконую та мене важко відволікти після погруження в роботу
0,13.12.2023 17:14:56,3,2,4,3,5,4,3,5,4,...,4,3,4,3,3,4,4,3,5,3
1,13.12.2023 17:21:03,2,3,1,4,3,5,3,4,2,...,3,2,3,2,2,3,2,4,3,4
2,13.12.2023 17:47:13,3,4,5,5,4,4,3,5,5,...,4,3,5,2,5,5,3,5,1,2
3,13.12.2023 18:03:34,1,1,5,1,1,5,1,1,1,...,5,2,1,1,1,1,1,3,1,1
4,13.12.2023 18:05:36,5,5,3,4,4,5,5,5,5,...,5,5,5,5,4,4,3,4,2,1
5,13.12.2023 18:09:50,1,1,1,4,5,5,3,1,5,...,1,1,4,1,4,2,3,4,3,3
6,13.12.2023 18:42:14,5,4,5,5,4,4,4,4,5,...,3,2,5,4,5,4,3,4,3,4
7,13.12.2023 19:18:19,5,2,3,4,3,2,3,3,3,...,2,3,3,1,2,3,5,2,3,4
8,13.12.2023 21:09:46,4,3,2,5,4,4,4,4,5,...,4,4,3,4,5,4,3,5,3,3
9,16.12.2023 11:36:37,4,2,4,3,2,4,2,3,2,...,2,3,4,3,4,3,4,3,4,3


In [288]:
df.columns

Index(['Позначка часу',
       'Коли я отримую подарунки, я більше ціную практичні та корисні речі, ніж щось декоративне',
       'Моя енергія ніколи не вичерпується',
       'Теоретичні ідеї та дилеми – це не те, на що варто витрачати багато часу\n(на скільки Ви погоджуєтесь з твердженням)',
       'Чи вам подобається займатися фізичною активністю та спортом? ',
       'Чи цікавить вас робота з різноманітними інструментами? ',
       'Чи вам подобається працювати на відкритому повітрі? ',
       'Чи вам цікаво розв'язувати складні математичні задачі? ',
       'Чи любите ви читати наукові книги або журнали? ',
       'Мені завжди цікаво почути унікальні точки зору на речі',
       'Мені подобається вивчати різноманітну інформацію про будь-яку тему',
       'Як ви відноситесь до вивчення нових технологій та програм? ',
       'Люди часто кажуть мені, що мої ідеї геніальні',
       'Немає такої форми мистецтва чи події, яку б я не хотів би дослідити/відвідати',
       'Мені важливий про

In [289]:
element_type = type(df["Чи вам подобається займатися фізичною активністю та спортом? "][0])

print(element_type)

<class 'str'>


In [290]:
numeric_columns = df.columns[1:]

df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

df

,Позначка часу,"Коли я отримую подарунки, я більше ціную практичні та корисні речі, ніж щось декоративне",Моя енергія ніколи не вичерпується,"Теоретичні ідеї та дилеми – це не те, на що варто витрачати багато часу\n(на скільки Ви погоджуєтесь з твердженням)",Чи вам подобається займатися фізичною активністю та спортом?,Чи цікавить вас робота з різноманітними інструментами?,Чи вам подобається працювати на відкритому повітрі?,Чи вам цікаво розв'язувати складні математичні задачі?,Чи любите ви читати наукові книги або журнали?,Мені завжди цікаво почути унікальні точки зору на речі,...,Я чудово вмію змусити інших підтримати мої ідеї\n\n,Я легко можу змінити думки оточуючих щодо якоїсь теми,Чи вам подобається приймати важливі рішення та вирішувати проблеми?,Чи берете ви участь у лідерських або підприємницьких ініціативах?,Чи вам подобається порядок і систематичність в роботі?,"Я завжди докладаю максимальних зусиль, щоб уникнути помилок\n\n",Чи легше вам працювати за чіткими правилами і встановленими стандартами?,Чи цікавитеся ви організаційними аспектами роботи?,"Я віддаю перевагу дотриманню правил, ніж імпровізації",Я легко концентруюсь на задачах які виконую та мене важко відволікти після погруження в роботу
0,13.12.2023 17:14:56,3,2,4,3,5,4,3,5,4,...,4,3,4,3,3,4,4,3,5,3
1,13.12.2023 17:21:03,2,3,1,4,3,5,3,4,2,...,3,2,3,2,2,3,2,4,3,4
2,13.12.2023 17:47:13,3,4,5,5,4,4,3,5,5,...,4,3,5,2,5,5,3,5,1,2
3,13.12.2023 18:03:34,1,1,5,1,1,5,1,1,1,...,5,2,1,1,1,1,1,3,1,1
4,13.12.2023 18:05:36,5,5,3,4,4,5,5,5,5,...,5,5,5,5,4,4,3,4,2,1
5,13.12.2023 18:09:50,1,1,1,4,5,5,3,1,5,...,1,1,4,1,4,2,3,4,3,3
6,13.12.2023 18:42:14,5,4,5,5,4,4,4,4,5,...,3,2,5,4,5,4,3,4,3,4
7,13.12.2023 19:18:19,5,2,3,4,3,2,3,3,3,...,2,3,3,1,2,3,5,2,3,4
8,13.12.2023 21:09:46,4,3,2,5,4,4,4,4,5,...,4,4,3,4,5,4,3,5,3,3
9,16.12.2023 11:36:37,4,2,4,3,2,4,2,3,2,...,2,3,4,3,4,3,4,3,4,3


In [291]:
element_type = type(df["Чи вам подобається займатися фізичною активністю та спортом? "][0])

print(element_type)

<class 'numpy.int64'>


In [292]:

numeric_columns = df.columns[1:]
riasec_columns = ['Realistic', 'Investigative', 'Artistic', 'Social', 'Enterprising', 'Conventional']
number_of_questions = 36
questions_per_section = 6
averages_df = pd.DataFrame()

for i in range(0, number_of_questions, questions_per_section):
    start_idx = i
    end_idx = i + questions_per_section
    selected_cols = df.columns[start_idx+1:end_idx]
    section_name = riasec_columns[int(end_idx/questions_per_section)-1]
    averages_df[section_name] = df[selected_cols].mean(axis=1) * (7/5)

final_df = pd.concat([df['Позначка часу'], averages_df], axis=1)
final_df_rm = final_df.where(final_df['Позначка часу'] == '13.12.2023 19:18:19')

final_df


,Позначка часу,Realistic,Investigative,Artistic,Social,Enterprising,Conventional
0,13.12.2023 17:14:56,4.76,5.04,4.48,3.64,4.48,5.32
1,13.12.2023 17:21:03,3.64,3.92,3.92,3.64,3.64,3.92
2,13.12.2023 17:47:13,5.88,6.44,6.65,7.00,5.60,5.32
3,13.12.2023 18:03:34,2.52,1.40,5.88,2.24,4.20,1.96
4,13.12.2023 18:05:36,5.88,7.00,5.04,4.76,7.00,4.76
5,13.12.2023 18:09:50,3.36,4.76,7.00,3.64,3.36,4.48
6,13.12.2023 18:42:14,6.44,5.88,3.64,3.64,4.76,5.32
7,13.12.2023 19:18:19,4.76,5.04,2.24,4.48,3.36,4.20
8,13.12.2023 21:09:46,5.04,6.16,6.44,5.88,5.88,5.60
9,16.12.2023 11:36:37,4.20,3.92,3.92,4.48,4.48,5.04


In [293]:
df_merged = pd.merge(final_df, df, on='Позначка часу')

In [294]:
df_merged

,Позначка часу,Realistic,Investigative,Artistic,Social,Enterprising,Conventional,"Коли я отримую подарунки, я більше ціную практичні та корисні речі, ніж щось декоративне",Моя енергія ніколи не вичерпується,"Теоретичні ідеї та дилеми – це не те, на що варто витрачати багато часу\n(на скільки Ви погоджуєтесь з твердженням)",...,Я чудово вмію змусити інших підтримати мої ідеї\n\n,Я легко можу змінити думки оточуючих щодо якоїсь теми,Чи вам подобається приймати важливі рішення та вирішувати проблеми?,Чи берете ви участь у лідерських або підприємницьких ініціативах?,Чи вам подобається порядок і систематичність в роботі?,"Я завжди докладаю максимальних зусиль, щоб уникнути помилок\n\n",Чи легше вам працювати за чіткими правилами і встановленими стандартами?,Чи цікавитеся ви організаційними аспектами роботи?,"Я віддаю перевагу дотриманню правил, ніж імпровізації",Я легко концентруюсь на задачах які виконую та мене важко відволікти після погруження в роботу
0,13.12.2023 17:14:56,4.76,5.04,4.48,3.64,4.48,5.32,3,2,4,...,4,3,4,3,3,4,4,3,5,3
1,13.12.2023 17:21:03,3.64,3.92,3.92,3.64,3.64,3.92,2,3,1,...,3,2,3,2,2,3,2,4,3,4
2,13.12.2023 17:47:13,5.88,6.44,6.65,7.00,5.60,5.32,3,4,5,...,4,3,5,2,5,5,3,5,1,2
3,13.12.2023 18:03:34,2.52,1.40,5.88,2.24,4.20,1.96,1,1,5,...,5,2,1,1,1,1,1,3,1,1
4,13.12.2023 18:05:36,5.88,7.00,5.04,4.76,7.00,4.76,5,5,3,...,5,5,5,5,4,4,3,4,2,1
5,13.12.2023 18:09:50,3.36,4.76,7.00,3.64,3.36,4.48,1,1,1,...,1,1,4,1,4,2,3,4,3,3
6,13.12.2023 18:42:14,6.44,5.88,3.64,3.64,4.76,5.32,5,4,5,...,3,2,5,4,5,4,3,4,3,4
7,13.12.2023 19:18:19,4.76,5.04,2.24,4.48,3.36,4.20,5,2,3,...,2,3,3,1,2,3,5,2,3,4
8,13.12.2023 21:09:46,5.04,6.16,6.44,5.88,5.88,5.60,4,3,2,...,4,4,3,4,5,4,3,5,3,3
9,16.12.2023 11:36:37,4.20,3.92,3.92,4.48,4.48,5.04,4,2,4,...,2,3,4,3,4,3,4,3,4,3


In [295]:
df_merged.fillna(0, inplace=True)

In [296]:
df_merged

,Позначка часу,Realistic,Investigative,Artistic,Social,Enterprising,Conventional,"Коли я отримую подарунки, я більше ціную практичні та корисні речі, ніж щось декоративне",Моя енергія ніколи не вичерпується,"Теоретичні ідеї та дилеми – це не те, на що варто витрачати багато часу\n(на скільки Ви погоджуєтесь з твердженням)",...,Я чудово вмію змусити інших підтримати мої ідеї\n\n,Я легко можу змінити думки оточуючих щодо якоїсь теми,Чи вам подобається приймати важливі рішення та вирішувати проблеми?,Чи берете ви участь у лідерських або підприємницьких ініціативах?,Чи вам подобається порядок і систематичність в роботі?,"Я завжди докладаю максимальних зусиль, щоб уникнути помилок\n\n",Чи легше вам працювати за чіткими правилами і встановленими стандартами?,Чи цікавитеся ви організаційними аспектами роботи?,"Я віддаю перевагу дотриманню правил, ніж імпровізації",Я легко концентруюсь на задачах які виконую та мене важко відволікти після погруження в роботу
0,13.12.2023 17:14:56,4.76,5.04,4.48,3.64,4.48,5.32,3,2,4,...,4,3,4,3,3,4,4,3,5,3
1,13.12.2023 17:21:03,3.64,3.92,3.92,3.64,3.64,3.92,2,3,1,...,3,2,3,2,2,3,2,4,3,4
2,13.12.2023 17:47:13,5.88,6.44,6.65,7.00,5.60,5.32,3,4,5,...,4,3,5,2,5,5,3,5,1,2
3,13.12.2023 18:03:34,2.52,1.40,5.88,2.24,4.20,1.96,1,1,5,...,5,2,1,1,1,1,1,3,1,1
4,13.12.2023 18:05:36,5.88,7.00,5.04,4.76,7.00,4.76,5,5,3,...,5,5,5,5,4,4,3,4,2,1
5,13.12.2023 18:09:50,3.36,4.76,7.00,3.64,3.36,4.48,1,1,1,...,1,1,4,1,4,2,3,4,3,3
6,13.12.2023 18:42:14,6.44,5.88,3.64,3.64,4.76,5.32,5,4,5,...,3,2,5,4,5,4,3,4,3,4
7,13.12.2023 19:18:19,4.76,5.04,2.24,4.48,3.36,4.20,5,2,3,...,2,3,3,1,2,3,5,2,3,4
8,13.12.2023 21:09:46,5.04,6.16,6.44,5.88,5.88,5.60,4,3,2,...,4,4,3,4,5,4,3,5,3,3
9,16.12.2023 11:36:37,4.20,3.92,3.92,4.48,4.48,5.04,4,2,4,...,2,3,4,3,4,3,4,3,4,3


In [302]:
X = df_merged.iloc[0:1, 7:43].values 
y = df_merged.iloc[0:1, 1:7].values
X

array([[3., 2., 4., 3., 5., 4., 3., 5., 4., 3., 3., 4., 3., 2., 4., 3.,
        4., 2., 3., 2., 3., 2., 3., 4., 3., 2., 4., 3., 4., 3., 3., 4.,
        4., 3., 5., 3.]])

In [303]:
y

array([[4.76, 5.04, 4.48, 3.64, 4.48, 5.32]])

In [304]:
X = df_merged.iloc[:, 7:37].values  
y = df_merged.iloc[:, 1:7].values

class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

class FFNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = FFNModel(input_size=X.shape[1], output_size=y.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


train_dataset = CustomDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

num_epochs = 250
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

test_dataset = CustomDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()
total_loss = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        print(outputs)
        loss = criterion(outputs, targets)
        total_loss += loss.item()

average_loss = total_loss / len(test_loader)
print(f'Average Loss on Test Data: {average_loss}')

Epoch 1/250, Loss: 26.16724967956543
Epoch 2/250, Loss: 24.876462936401367
Epoch 3/250, Loss: 23.627973556518555
Epoch 4/250, Loss: 22.410921096801758
Epoch 5/250, Loss: 21.224924087524414
Epoch 6/250, Loss: 20.066640853881836
Epoch 7/250, Loss: 18.91691017150879
Epoch 8/250, Loss: 17.773710250854492
Epoch 9/250, Loss: 16.643091201782227
Epoch 10/250, Loss: 15.525934219360352
Epoch 11/250, Loss: 14.413753509521484
Epoch 12/250, Loss: 13.32833194732666
Epoch 13/250, Loss: 12.267650604248047
Epoch 14/250, Loss: 11.226190567016602
Epoch 15/250, Loss: 10.209699630737305
Epoch 16/250, Loss: 9.230077743530273
Epoch 17/250, Loss: 8.289762496948242
Epoch 18/250, Loss: 7.392576217651367
Epoch 19/250, Loss: 6.541627883911133
Epoch 20/250, Loss: 5.743217945098877
Epoch 21/250, Loss: 5.004154205322266
Epoch 22/250, Loss: 4.327071666717529
Epoch 23/250, Loss: 3.7173619270324707
Epoch 24/250, Loss: 3.176784038543701
Epoch 25/250, Loss: 2.709153652191162
Epoch 26/250, Loss: 2.315061092376709
Epoch 27

In [308]:

df.fillna(0, inplace=True)



input_tensor = torch.tensor(X, dtype=torch.float32)

if len(input_tensor.shape) == 1:
    input_tensor = input_tensor.unsqueeze(0)

with torch.no_grad():
    predictions = model(input_tensor)

columns = ['R', 'I', 'A', 'S', 'E', 'C']
df_predictions = pd.DataFrame(predictions.numpy(), columns=columns)

df_predictions.insert(0, 'Позначка часу', df['Позначка часу'])
df_predictions.iloc[:, 1:] = df_predictions.iloc[:, 1:].applymap(lambda x: min(x, 7))
df_predictions

C:\Users\yg_ya\AppData\Local\Temp\ipykernel_2432\141191465.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_predictions.iloc[:, 1:] = df_predictions.iloc[:, 1:].applymap(lambda x: min(x, 7))


,Позначка часу,R,I,A,S,E,C
0,13.12.2023 17:14:56,4.778446,5.071712,4.129416,4.581790,4.555662,4.468200
1,13.12.2023 17:21:03,3.356693,3.941606,3.871395,3.486937,3.764305,3.809934
2,13.12.2023 17:47:13,5.999536,6.181489,6.095975,6.471297,5.876038,5.353434
3,13.12.2023 18:03:34,2.343382,1.827418,5.616811,2.312938,3.890804,1.952389
4,13.12.2023 18:05:36,5.840718,6.754934,5.414213,5.043657,6.671498,5.372385
5,13.12.2023 18:09:50,3.639551,4.492902,6.659629,3.558465,3.679855,4.482189
6,13.12.2023 18:42:14,5.969301,6.163733,4.374558,4.514141,4.691145,5.284812
7,13.12.2023 19:18:19,4.474732,5.038565,2.649272,4.519227,3.222739,4.022819
8,13.12.2023 21:09:46,5.412952,6.315464,6.329535,5.533393,5.921116,5.494795
9,16.12.2023 11:36:37,4.599771,4.586858,3.940316,4.439301,4.441696,3.814388


In [301]:
final_df

,Позначка часу,Realistic,Investigative,Artistic,Social,Enterprising,Conventional
0,13.12.2023 17:14:56,4.76,5.04,4.48,3.64,4.48,5.32
1,13.12.2023 17:21:03,3.64,3.92,3.92,3.64,3.64,3.92
2,13.12.2023 17:47:13,5.88,6.44,6.65,7.00,5.60,5.32
3,13.12.2023 18:03:34,2.52,1.40,5.88,2.24,4.20,1.96
4,13.12.2023 18:05:36,5.88,7.00,5.04,4.76,7.00,4.76
5,13.12.2023 18:09:50,3.36,4.76,7.00,3.64,3.36,4.48
6,13.12.2023 18:42:14,6.44,5.88,3.64,3.64,4.76,5.32
7,13.12.2023 19:18:19,4.76,5.04,2.24,4.48,3.36,4.20
8,13.12.2023 21:09:46,5.04,6.16,6.44,5.88,5.88,5.60
9,16.12.2023 11:36:37,4.20,3.92,3.92,4.48,4.48,5.04
